# NYC Apartment Search

_[Project prompt](https://docs.google.com/document/d/1ogme9BJeHb2IZ6UREavUorF--nnxoWCYAAi8AZ4Q5jQ/edit?usp=sharing) and [grading rubric](https://docs.google.com/document/d/1XI9Yq_e-U-D3iH4jTPAtNteeP2Q9mtJ9NKbePWKeN_g/edit?usp=sharing)

_This scaffolding notebook may be used to help setup your final project. It's **totally optional** whether you make use of this or not._

_If you do use this notebook, everything provided is optional as well - you may remove or add code as you wish._

_**All code below should be consider "pseudo-code" - not functional by itself, and only an idea of a possible approach.**_

## Setup

In [1]:
!pip install geoalchemy2
!pip install geopandas

In [29]:
# All import statements needed for the project, for example:
import json
import pathlib
import urllib.parse
import os
import geoalchemy2 as gdb
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import requests
import shapely
import sqlalchemy as db
from shapely.geometry import Point
from sqlalchemy.orm import declarative_base

In [30]:
# Where data files will be read from/written to - this should already exist
DATA_DIR = pathlib.Path("/Users/catherinewang/Library/CloudStorage/GoogleDrive-xw2925@columbia.edu/My Drive/Final Project 4501/data")
ZIPCODE_DATA_FILE = DATA_DIR / "nyc_zipcodes" / "nyc_zipcodes.shp"
ZILLOW_DATA_FILE = DATA_DIR / "zillow_rent_data.csv"

NYC_DATA_APP_TOKEN = "1DrbcO2jXtETwL4T7Hm2ER2Lq"
BASE_NYC_DATA_URL = "https://data.cityofnewyork.us/resource/"
NYC_DATA_311 = "erm2-nwe9.geojson"
NYC_DATA_TREES = "pi5s-9p35.geojson"

In [4]:
# DB_NAME = "4501_Data_Analysis"
# DB_USER = "user1"
# DB_URL = f"postgres+psycopg2://{DB_USER}@localhost/{DB_NAME}"
# DB_SCHEMA_FILE = "schema.sql"
# # directory where DB queries for Part 3 will be saved
# QUERY_DIR = pathlib.Path("queries")

In [5]:
# Make sure the QUERY_DIRECTORY & DATA_DIR exists
if not DATA_DIR.exists():
    DATA_DIR.mkdir()
if not QUERY_DIR.exists():
    QUERY_DIR.mkdir()

## Part 1: Data Preprocessing

In [31]:
def download_nyc_geojson_data(url, force=False):
    # Parse the given URL
    parsed_url = urllib.parse.urlparse(url)
    url_path = parsed_url.path.strip("/")

    # Construct the filename based on the URL path
    filename = DATA_DIR / url_path

    # Check if the directory exists, and create it if it does not
    if not filename.parent.exists():
        os.makedirs(filename.parent, exist_ok=True)

    # Download the data if force is True or if the file does not exist
    if force or not filename.exists():
        print(f"Downloading {url} to {filename}...")
        try:
            # Request the data from the URL
            response = requests.get(url)
            response.raise_for_status()  # Raises a HTTPError if the response was an error

            # Save the data to the file
            with open(filename, "wb") as f:
                f.write(response.content)
            print(f"Done downloading {url}.")
        except Exception as e:
            print(f"Failed to download {url}. Error: {e}")
            return None
    else:
        print(f"File already exists at {filename}.")

    return filename

In [ ]:
def load_and_clean_zipcodes(ZIPCODE_DATA_FILE):
    # Load the shapefile
    gdf = gpd.read_file(ZIPCODE_DATA_FILE)

    # Drop rows with missing values
    gdf = gdf.dropna()

    # List of columns to keep
    columns_to_keep = ['ZIPCODE', 'geometry']
    gdf = gdf[columns_to_keep]

    # Remove duplicate ZIP codes
    gdf = gdf.drop_duplicates(subset='ZIPCODE')

    # Filter out ZIP codes that are not in NYC
    valid_nyc_zip_codes = (
        [f"{z:05d}" for z in range(10001, 10293)] +  # Manhattan
        [f"{z:05d}" for z in range(10451, 10476)] +  # Bronx
        [f"{z:05d}" for z in range(11201, 11257)] +  # Brooklyn
        [f"{z:05d}" for z in range(11001, 11698)] +  # Queens
        [f"{z:05d}" for z in range(10301, 10315)]    # Staten Island
    )
    gdf = gdf[gdf['ZIPCODE'].isin(valid_nyc_zip_codes)]

    # Cast columns to appropriate types
    gdf['ZIPCODE'] = gdf['ZIPCODE'].astype(str)

    # Save the cleaned GeoDataFrame back to a new shapefile
    cleaned_file_path = ZIPCODE_DATA_FILE.parent / "nyc_zipcodes_cleaned.shp"
    gdf.to_file(cleaned_file_path)

    return cleaned_file_path


In [19]:
def download_and_clean_311_data():
    limit = 1000
    offset = 0
    all_gdf = gpd.GeoDataFrame()

    while True:
        query = f"?$where=created_date >= '2022-02-01T00:00:00' AND created_date <= '2024-02-29T00:00:00' AND latitude IS NOT NULL&$limit={limit}&$offset={offset}&$$app_token={NYC_DATA_APP_TOKEN}"
        full_url = f"{BASE_NYC_DATA_URL}{NYC_DATA_311}{query}"

        filename = download_nyc_geojson_data(full_url, True)
        if filename is None:
            print(f"Failed to download data for offset {offset}.")
            break

        gdf = gpd.read_file(filename)

        if gdf.empty:
            print("No more data to process.")
            break

        gdf.columns = [col.lower().replace(' ', '_') for col in gdf.columns]

        necessary_columns = ['unique_key', 'created_date', 'incident_zip', 'complaint_type','city', 'geometry']

        gdf = gdf[necessary_columns].dropna(subset=['incident_zip', 'city'])

        gdf['created_date'] = pd.to_datetime(gdf['created_date'])

        all_gdf = pd.concat([all_gdf, gdf], ignore_index=True)

        offset += limit
        break
    if not all_gdf.empty:
        all_gdf.set_crs(epsg=4326, inplace=True)
        geojson_path = DATA_DIR / 'nyc_311_data_cleaned.geojson'
        all_gdf.to_file(geojson_path, driver='GeoJSON')
        print(f"Data saved to {geojson_path}")

    return all_gdf

In [20]:
clean_311_nyc_data = download_and_clean_311_data()

Done downloading https://data.cityofnewyork.us/resource/erm2-nwe9.geojson?$where=created_date >= '2022-02-01T00:00:00' AND created_date <= '2024-02-29T00:00:00' AND latitude IS NOT NULL&$limit=1000&$offset=0&$$app_token=1DrbcO2jXtETwL4T7Hm2ER2Lq.
Data saved to /Users/catherinewang/Library/CloudStorage/GoogleDrive-xw2925@columbia.edu/My Drive/Final Project 4501/data/nyc_311_data_cleaned.geojson


In [32]:
def download_and_clean_tree_data():
    limit = 1000
    offset = 0
    all_trees_gdf = gpd.GeoDataFrame(columns=['tree_id', 'zipcode', 'geometry', 'status', 'health', 'spc_latin', 'zip_city'])  # Initialize with columns
    all_trees_gdf.set_crs(epsg=4326, inplace=True)  # Set CRS to WGS 84 from the start

    while True:
        query = f"?$where=latitude IS NOT NULL&$limit={limit}&$offset={offset}&$$app_token={NYC_DATA_APP_TOKEN}"
        full_url = f"https://data.cityofnewyork.us/resource/uvpi-gqnh.geojson{query}"
        filename = download_nyc_geojson_data(full_url, True)  # Assume this function returns the filepath correctly

        if not filename:
            break  # Exit the loop if no filename is returned

        gdf_batch = gpd.read_file(filename)

        if gdf_batch.empty:
            print("No more data to process.")
            break

        gdf_batch.columns = [col.lower().replace(' ', '_') for col in gdf_batch.columns]

        # Filter for necessary columns, including latitude and longitude for geometry creation
        columns_to_keep = ['tree_id', 'zipcode', 'latitude', 'longitude', 'status', 'health', 'spc_latin', 'zip_city']
        gdf_batch = gdf_batch[columns_to_keep].dropna(subset=['zipcode', 'zip_city'])

        # Convert latitude and longitude to float
        gdf_batch['latitude'] = gdf_batch['latitude'].astype(float)
        gdf_batch['longitude'] = gdf_batch['longitude'].astype(float)

        # Create geometry column from latitude and longitude
        gdf_batch['geometry'] = [Point(xy) for xy in zip(gdf_batch.longitude, gdf_batch.latitude)]
        # Drop the latitude and longitude columns as they are no longer needed
        gdf_batch.drop(columns=['latitude', 'longitude'], inplace=True)
        # Create a GeoDataFrame with the geometry column
        gdf_batch = gpd.GeoDataFrame(gdf_batch, geometry='geometry')
        gdf_batch.set_crs(epsg=4326, inplace=True)  # Ensure the GeoDataFrame is in WGS 84 CRS

        # Concatenate with the main GeoDataFrame
        all_trees_gdf = pd.concat([all_trees_gdf, gdf_batch], ignore_index=True)

        offset += limit  # Increment the offset for the next batch
        break
    if not all_trees_gdf.empty:
        # Save the cleaned GeoDataFrame to a GeoJSON file, now that geometry is used
        cleaned_filepath = DATA_DIR / 'nyc_trees_data_cleaned.geojson'
        all_trees_gdf.to_file(cleaned_filepath, driver='GeoJSON')
        print(f"Cleaned tree data saved to {cleaned_filepath}")

    return all_trees_gdf


In [33]:
clean_tree_nyc_data = download_and_clean_tree_data()

Done downloading https://data.cityofnewyork.us/resource/uvpi-gqnh.geojson?$where=latitude IS NOT NULL&$limit=1000&$offset=0&$$app_token=1DrbcO2jXtETwL4T7Hm2ER2Lq.
Cleaned tree data saved to /Users/catherinewang/Library/CloudStorage/GoogleDrive-xw2925@columbia.edu/My Drive/Final Project 4501/data/nyc_trees_data_cleaned.geojson


In [3]:
def load_and_clean_zillow_data(NYC_Cities):
    # Load the data into a DataFrame
    zillow_rent_df = pd.read_csv(ZILLOW_DATA_FILE)
    
    # Assuming the date columns are in a format like 'YYYY-MM-DD'
    # First, identify all columns that are not date columns but need to be kept (e.g., 'RegionName', 'City')
    non_date_columns = ['RegionName', 'City']

    # Dynamically generate a list of date columns to keep, based on the specified date range
    # Convert all column names to datetime where possible; this will yield NaT for non-date columns
    date_columns = pd.to_datetime(zillow_rent_df.columns, errors='coerce', format='%Y-%m-%d')

    # Identify columns within the specified date range
    start_date = pd.to_datetime('2022-02-01')
    end_date = pd.to_datetime('2024-01-31')
    date_columns_to_keep = zillow_rent_df.columns[(date_columns >= start_date) & (date_columns <= end_date)]

    # Combine the non-date columns to keep with the date columns identified in the range
    columns_to_keep = non_date_columns + date_columns_to_keep.tolist()

    # Filter the DataFrame to include only the identified columns
    zillow_rent_df = zillow_rent_df[columns_to_keep]

    # Filter rows where 'City' is in the NYC_Cities list
    zillow_rent_df = zillow_rent_df[zillow_rent_df['City'].str.lower().isin([city.lower() for city in NYC_Cities])]

    return zillow_rent_df.dropna()

In [ ]:
def load_all_data():
    geodf_zipcode_data = load_and_clean_zipcodes(ZIPCODE_DATA_FILE)
    geodf_311_data = download_and_clean_311_data()
    geodf_tree_data = download_and_clean_tree_data()
    df_zillow_data = load_and_clean_zillow_data()
    return (
        geodf_zipcode_data,
        geodf_311_data,
        geodf_tree_data,
        df_zillow_data
    )

In [ ]:
geodf_zipcode_data, geodf_311_data, geodf_tree_data, df_zillow_data = load_all_data()

In [ ]:
# Show basic info about each dataframe
geodf_zipcode_data.info()

In [ ]:
# Show first 5 entries about each dataframe
geodf_zipcode_data.head()

In [ ]:
geodf_311_data.info()

In [ ]:
geodf_311_data.head()

In [ ]:
geodf_tree_data.info()

In [ ]:
geodf_tree_data.head()

In [ ]:
df_zillow_data.info()

In [ ]:
df_zillow_data.head()

## Part 2: Storing Data

In [ ]:
def setup_new_postgis_database(username, db_name):
    raise NotImplementedError()

In [ ]:
setup_new_postgis_database(DB_USER, DB_NAME)

### Creating Tables


These are just a couple of options to creating your tables; you can use one or the other, a different method, or a combination.

In [ ]:
engine = db.create_engine(DB_URL)

#### Option 1: SQL

In [ ]:
# If using SQL (as opposed to SQLAlchemy), define the SQL statements to create your 4 tables.
# You may be creating more tables depending on how you're setting up your constraints/relationships
# or if you're completing the extra credit.
ZIPCODE_SCHEMA = """
TODO
"""

ZIPCODE_G

NYC_311_SCHEMA = """
TODO
"""

NYC_TREE_SCHEMA = """
TODO
"""

ZILLOW_SCHEMA = """
TODO
"""

In [ ]:
# create that required schema.sql file
with open(DB_SCHEMA_FILE, "w") as f:
    f.write(ZIPCODE_SCHEMA)
    f.write(NYC_311_SCHEMA)
    f.write(NYC_TREE_SCHEMA)
    f.write(ZILLOW_SCHEMA)

In [ ]:
# If using SQL (as opposed to SQLAlchemy), execute the schema files to create tables
with engine.connect() as connection:
    pass

#### Option 2: SQLAlchemy

In [ ]:
Base = declarative_base()

class Tree(Base):
    __tablename__ = "trees"

    ...


In [ ]:
Base.metadata.create_all(engine)

### Add Data to Database

These are just a couple of options to write data to your tables; you can use one or the other, a different method, or a combination.

#### Option 1: SQL

In [ ]:
def write_dataframes_to_table(tablename_to_dataframe):
    # write INSERT statements or use pandas/geopandas to write SQL
    raise NotImplemented()

In [ ]:
tablename_to_dataframe = {
    "zipcodes": geodf_zipcode_data,
    "complaints": geodf_311_data,
    "trees": geodf_tree_data,
    "rents": df_zillow_data,
}

In [ ]:
write_dataframes_to_table(tablename_to_dataframe)

#### Option 2: SQLAlchemy

In [ ]:
Session = db.orm.sessionmaker(bind=engine)
session = Session()

In [ ]:
for row in geodf_tree_data.iterrows():
    tree = Tree(...)
    session.add(tree)

In [ ]:
session.commit()

## Part 3: Understanding the Data

### Query 1

In [ ]:
# Helper function to write the queries to file
def write_query_to_file(query, outfile):
    raise NotImplementedError()

In [ ]:
QUERY_1_FILENAME = QUERY_DIR / "FILL_ME_IN"

QUERY_1 = """
FILL_ME_IN
"""

In [ ]:
with engine.connect() as conn:
    result = conn.execute(db.text(QUERY_1))
    for row in result:
        print(row)

In [ ]:
write_query_to_file(QUERY_1, QUERY_1_FILENAME)

## Part 4: Visualizing the Data

### Visualization 1

In [ ]:
# use a more descriptive name for your function
def plot_visual_1(dataframe):
    figure, axes = plt.subplots(figsize=(20, 10))
    
    values = "..."  # use the dataframe to pull out values needed to plot
    
    # you may want to use matplotlib to plot your visualizations;
    # there are also many other plot types (other 
    # than axes.plot) you can use
    axes.plot(values, "...")
    # there are other methods to use to label your axes, to style 
    # and set up axes labels, etc
    axes.set_title("Some Descriptive Title")
    
    plt.show()

In [ ]:
def get_data_for_visual_1():
    # Query your database for the data needed.
    # You can put the data queried into a pandas/geopandas dataframe, if you wish
    raise NotImplementedError()

In [ ]:
some_dataframe = get_data_for_visual_1()
plot_visual_1(some_dataframe)